<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Traducció**

---
Exemple d’una tasca de tipus seqüència a seqüència (sequence-to-sequence o seq2seq).

**Què és una tasca seq2seq?**

És un problema on:

L’entrada és una seqüència (per exemple, una frase o paràgraf). La sortida també és una seqüència, però transformada d’alguna manera.

**Exemple principal: Traducció**

* Entrada: "Bon dia, com estàs?"

* Sortida: "Good morning, how are you?"

El model aprèn a convertir una seqüència de text en una altra, mantenint el significat.

**Altres aplicacions relacionades:**

1. Resum (summarization): Reduir un text llarg a un de més curt mantenint-ne les idees clau.

1. Canvi d’estil (style transfer): Convertir el to d’un text (per exemple, de formal a informal).

1. Resposta generativa a preguntes: Donar una resposta nova a partir d’un context i una pregunta (no només escollir-la com en classificació múltiple).


Es proposa de com entrenar o ajustar (fine-tune) un model de traducció automàtica, i compara dues opcions:

1. **Entrenar des de zero** (from scratch)

Necessites un corpus molt gran de textos traduïts entre dues (o més) llengües.

És el procés que es fa, per exemple, quan s’entrena un model com mT5 o mBART des del principi. És costós i lent, perquè el model parteix sense cap coneixement.

1. **Fine-tuning d’un model ja entrenat** (reentrenar-lo parcialment)

Agafes un model ja entrenat, com mT5, mBART o Marian, que ja sap fer traducció.
L’ajustes amb el teu corpus específic, per exemple, textos d’un domini concret (com aplicacions KDE). És molt més ràpid i eficient, perquè el model ja sap molt de la llengua i només ha de refinar el seu comportament.

# **Preparar les dades**

---
Per entrenar o ajustar un model de traducció, el primer pas essencial és tenir un conjunt de dades adequat. Això vol dir:

1. **Dades en forma de parelles de frases**.

Necessites frases emparellades: una en la llengua d’origen i una altra en la llengua de destinació.

**Exemple:**

* Anglès: "How are you?"

* Francès: "Comment ça va ?"

Aquestes parelles són les que el model utilitza per aprendre com transformar una frase d’una llengua a una altra.

2. **Exemple amb el conjunt de dades KDE4**

És un corpus públic format per fitxers de programari (els diàlegs, menús, etc.) traduïts entre idiomes per al projecte KDE (programari de codi obert). Aquest conjunt ja té les parelles preparades: una frase en anglès i la seva traducció (per exemple, al francès).

3. **Utilitzar el teu propi corpus**

Disposant de textos bilingües (per exemple, traduccions d’un llibre, documents oficials, webs...), pots fer servir aquestes dades. El més important és que siguin parelles alineades (una frase i la seva traducció exacta).


In [ ]:
!pip install datasets
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

# Tenim 210.173 parells de frases, però en una sola divisió,
# per tant haurem de crear el nostre propi conjunt de validació

In [ ]:
raw_datasets

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")
split_datasets["train"][1]["translation"]

**Conceptes clau:**

1. **El dataset** conté traduccions completes : El corpus KDE4 té frases tècniques de programari. Totes les paraules estan traduïdes literalment al francès, incloses les tècniques.

  * **Exemple**: "threads" (anglès) → "fils de discussion" (francès).

2. **L’ús real** del llenguatge tècnic pot ser diferent. En converses reals, els enginyers francesos no sempre tradueixen els termes tècnics. Solen deixar paraules com "thread", "bug", o "CPU" en anglès, fins i tot quan parlen en francès.

3. **El model** preentrenat pot triar la forma més natural. Com que el model preentrenat ha vist moltes frases de francès real, sovint ha après a deixar "threads" sense traduir.

Quan es fa fine-tuning amb un dataset com KDE4 (on threads → fils de discussion), s'està ensenyant-li una versió més normativa o “correcta” de la traducció.

In [ ]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [ ]:
split_datasets["train"][10]["translation"]

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

**El fine-tuning** no requereix un dataset propi, però el que sí que fa és personalitzar el comportament del model mitjançant l’exposició a un conjunt de dades més específic que el corpus massiu del preentrenament.

# **Processant les dades**

---
Abans de fer servir textos amb un model, cal convertir-los en tokens.

1. **Els models no entenen text en brut**. Els models com Marian, BERT o GPT no processen directament el text. Necessiten que el text s’hagi convertit en nombres enters que representen paraules o fragments de paraula. Aquest procés es diu tokenització → converteix el text en IDs de tokens.

2. **Tokenitzar tant els inputs com les targets**. En una tasca de traducció:

  * Input: frase en anglès.

  * Target: frase en francès.

3. **Cal tokenitzar les dues parts perquè el model pugui**:

  * Llegir la frase d’origen.

  * Aprendre a generar la frase de destinació.

4. **Crear l’objecte tokenitzador**. Es fa servir un tokenizer específic vinculat al model Marian que s’està emprant. Si canvies d’idiomes (per exemple, alemany a italià), has de canviar el tokenizer i el model, triant un checkpoint diferent.

5. **Helsinki-NLP ofereix molts models**. L’organització Helsinki-NLP (a Hugging Face) ha publicat més de mil models de traducció per a diferents parells de llengües. Aquests models es poden carregar fàcilment amb Hugging Face, indicant el nom correcte del model (checkpoint).


In [10]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

return_tensors="pt" retorna:

1. **input_ids**: cada número representa un token.

1. **attention_mask**: posa 1 a les posicions útils (paraules reals) i 0 als espais de farciment (padding).

In [ ]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

Com podem veure, la sortida conté els ID d'entrada associats a la frase anglesa, mentre que els ID associats a la francesa s'emmagatzemen al camp 'labels'. Si oblideu indicar que esteu tokenitzant etiquetes, seran tokenitzades pel tokenitzador d'entrada, que en el cas d'un model marià no anirà gens bé.

In [ ]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

Com podem veure, l'ús del tokenizer anglès per preprocessar una frase francesa dóna com a resultat molts més fitxes, ja que el tokenizer no coneix cap paraula francesa (excepte les que també apareixen en anglès, com ara "discussion").

Com que inputs és un diccionari amb les nostres claus habituals (ID d'entrada, màscara d'atenció, etc.), l'últim pas és definir la funció de preprocessament que aplicarem als conjunts de dades:

In [13]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

1. **split_datasets.map(...)**

Aplica una funció de preprocessament a cada element del dataset (funciona com map() de Python però optimitzat per grans volums de dades). split_datasets conté els tres subconjunts (train, validation, test).

2. **preprocess_function**

És una funció que tu has definit abans, que agafa una entrada (frase original i traducció) i la tokenitza. Potser també crea la labels (frase de destinació en forma de tokens), que el model utilitzarà per aprendre.

3. **batched=True**

Indica que la funció rebrà un lot (batch) de dades en lloc d’una sola instància.
Això és molt més ràpid i eficient.

4. **remove_columns=split_datasets["train"].column_names**

Elimina les columnes originals (ex: "en", "fr") després de tokenitzar. Això evita que hi hagi columnes duplicades o innecessàries.

# **Fine-tuning el model amb el Trainer API**

---
Aquest fragment de codi fa dues coses importants: com carregar un model preparat per a seqüències a seqüències (seq2seq) i com utilitzar el Seq2SeqTrainer, que és una versió especialitzada del Trainer de Hugging Face pensada per a tasques com la traducció, el resum o el question answering generatiu.

1. **Què fa això?**

Carrega un model preentrenat pensat per a tasques de seqüència a seqüència amb llenguatge natural (Seq2SeqLM vol dir Sequence-to-Sequence Language Modeling).

  * **Exemple** de models compatibles: MarianMT, mBART, T5, BART

2. **Per què aquest model funciona ja?**

Perquè ja està entrenat prèviament per traduir, per exemple de l’anglès al francès. No cal crear cap capa nova ni inicialitzar pesos: ja sap traduir, només es vol afinar amb el corpus.




# **Col·lecció de dades**

---

